- <span style="color:black">Занятие 1 (NLP)</span>
- <span style="color:blue">**Занятие 2 (NLP)**</span>
- <span style="color:gray">Занятие 3 (NLP)</span>
- <span style="color:gray">Занятие 4 (Recommender Systems)</span>
- <span style="color:gray">Занятие 5 (Recommender Systems)</span>
- <span style="color:gray">Занятие 6 (CRISP-DM)</span>
- <span style="color:gray">Занятие 7 (to be anounced)</span>
- <span style="color:gray">Занятие 8 (to be anounced)</span>
- <span style="color:gray">Занятие 9 (Class Hours)</span>
- <span style="color:gray">Занятие 10 (Class Hours)</span>



## Review предыдущего занятия

Тексты можно использовать в задачах машинного обучения точно так же, как обычные числовые данные. Единственное требование - нужно как-то представить их в виде числового вектора.

- <span style="color:blue">Document</span> - единица текста, соотвествующая одному наблюдению (статья, отзыв, предложение)
- <span style="color:blue">Corpus</span> - набор всех документов, которыми мы располагаем
- <span style="color:blue">Term</span> или <span style="color:blue">Token</span> - составная часть, на которую бъется документ (например, слово)
- <span style="color:blue">Document-Term</span> матрица - матрица, у которой по строкам отложены документы, а по столбцам термы, значением же является частота данного терма в документе (или другие характеристики). Также иногда работают с ее транспонриованным вариантом Term-Document matrix.
- <span style="color:blue">Vocabulary</span> - набор всех термов из корпуса и назначенных им порядковых номеров
- <span style="color:blue">Vector-Space Model</span> - способ представления терма в виде многомерного вектора
- <span style="color:blue">Term Frequency</span> - частота терма в данном документе
- <span style="color:blue">Document Frequency</span> - в каком проценте документов корпуса встречается данный терм
- <span style="color:blue">Inverse Docuemnt Frequency</span> - 1/DF, коэффициент "уникальности" терма

В библиотеке Sklearn есть готовый датасет для работы с текстами (fetch_20newsgroups()), а также 3 класса для векторизации текстов: 
- CountVectorizer() - для простоты и скорости работы
- HashingVectorizer() - для эффективной работы с часто обновляющимся словарем
- TfidfVectorizer() - для улучшения качества предсказания предиктивных моделей, работающих с этими текстами

Также есть трансформация TfidfTransfomer(), которую можно применять отдельно над уже переведенным в векторное представление текстом для перевзвешивания признаков.

Типичный стек преобразований текста:
- нормализация (удаление неинформативных символов, цифр)
- токенизация (разделение текста на смысловые единицы)
- лемматизация (приведение слова к нормальной форме, именительный падеж, единственное число)
- стемминг (выделение основы слова)
- выделение частей речи
- фильтрация стоп-слов (соединительных слов, союзов и предлогов, специфичных для доменной области неинформативных слов)

POS_tagging - задача определения частей речи каждого слова



На прошлом занятии обсуждали важность TF-IDF преобразования. Данный принцип можно сформулировать в более общем виде: при сравнении двух объектов совпадение "редких" атрибутов гораздо больше говорит о схожести этих объектов, чем совпадение часто втсречающихся атрибутов.

Посмотрим как tf-idf взвешивание может влиять на точность модели классификации. Добавим в сетку параметров два значения: tf-idf включено/выключено

## О важности TF-IDF преобразования

На прошлом занятии обсуждали важность TF-IDF преобразования. Данный принцип можно сформулировать в более общем виде: при сравнении двух векторов совпадение "редких" элементов гораздо больше говорит о схожести этих объектов, чем совпадение часто втсречающихся атрибутов.

Посмотрим как tf-idf взвешивание может влиять на точность модели классификации. Добавим в сетку параметров два значения: tf-idf включено/выключено

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import fetch_20newsgroups

# Load datasets
categories_to_load = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
text_train = fetch_20newsgroups(subset='train', categories = categories_to_load, shuffle=True)
text_test = fetch_20newsgroups(subset='test', categories = categories_to_load, shuffle=True)

# Define classification pipeline (vectorizer + classifier)
text_clf = Pipeline([
        ('vect', TfidfVectorizer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None))])

# Define parameters grid
parameters = {
     'vect__use_idf': (True, False)
}

cvGrid = GridSearchCV(text_clf, parameters, cv=5)

cvGrid.fit(text_train.data, text_train.target)

# When fitting is done, study the structure of "cv_result_" dictionary
print(cvGrid.cv_results_)

# Print values of scoring metric for each parameter combination
print(cvGrid.cv_results_['mean_test_score'])           

{'mean_fit_time': array([0.47783971, 0.38655   ]), 'std_fit_time': array([0.11315002, 0.01208689]), 'mean_score_time': array([0.10601983, 0.08410416]), 'std_score_time': array([0.03947273, 0.00399529]), 'param_vect__use_idf': masked_array(data=[True, False],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'vect__use_idf': True}, {'vect__use_idf': False}], 'split0_test_score': array([0.96460177, 0.9159292 ]), 'split1_test_score': array([0.96238938, 0.8960177 ]), 'split2_test_score': array([0.96230599, 0.92904656]), 'split3_test_score': array([0.96452328, 0.9135255 ]), 'split4_test_score': array([0.97339246, 0.93569845]), 'mean_test_score': array([0.96544258, 0.91804348]), 'std_test_score': array([0.00409666, 0.01373159]), 'rank_test_score': array([1, 2], dtype=int32)}
[0.96544258 0.91804348]


Видим, что при наличии tf-idf взвешивания точность предсказательной модели выросла почти на 4 процента, что для задач машинного обучения очень существенно

## Сравнение текстовых векторов

Видим, что при наличии tf-idf взвешивания точность предсказательной модели выросла почти на 4 процента, что для задач машинного обучения очень существенно

Итак, мы научились представлять тексты в виде векторов в пространстве слов. Как их сравнивать между собой.

Чтобы оценить "схожесть" двух текстов чаще всего используются два расстояния:
- [косинусное расстояние](https://en.wikipedia.org/wiki/Cosine_similarity) (cosine distance)

    Косинусное расстояние может быть посчитано для любой пары векторов в исследуемом пространстве.
    
    $$\text{similarity} = \cos(\theta) = {\mathbf{A} \cdot \mathbf{B} \over \|\mathbf{A}\| \|\mathbf{B}\|} = \frac{ \sum\limits_{i=1}^{n}{A_i  B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{A_i^2}}  \sqrt{\sum\limits_{i=1}^{n}{B_i^2}} }$$
    
    <img src="img/cosine_plot.png" width=300>


- [мера Жаккара](https://en.wikipedia.org/wiki/Jaccard_index) (Jaccard index)

    <img src="img/jaccard_plot.jpg" width=350>
    
    Эта мера выводит число слов, которые присутствуют и в тексте A, и в тексте B, поделенное на число слов, которые есть или в тексте A, или в тексте B (то есть общее число слов в двух текстах).

    Мера Жаккара считается на множествах, поэтому может быть применена только к бинарным векторам (любой бинарный вектор представляет некоторое множество, например {1,0,0,1,0}, где 1 означает, что этот элемент входит в множество, а 0 - не входит).
    
    Таким образом, если строка описывается не бинарным векторным представлением (где каждый элемент означает, что слово входит или не входит), а например частотами слов, этот вектор надо сначала бинаризовать.

В математике многие метрики представляют в двух вариантах:
- близость (similarity), когда меняется от 0.0 (полное несовпадение) до 1.0 (полное совпадение)
- расстояние (distance), когда меняется от 0.0 (полное совпадение) до 1.0 (полное несовпадение)

Одно легко получается из другого вычитанием: similarity = 1 - distance. При чтении материалов важно не путать, о каком из двух вариантов метрики идет речь.

### Реализация в Sklearn

#### Cosine

В Sklearn можно посчитать косинусное расстояние (оба варианта метрики). Для этого в модуле <b>metrics.pairwise</b> есть функции 
- cosine_similarity(X,Y)
- cosine_distances(X,Y)

Эти функции умеют считать расстояния не только между парой векторов, но и могут вывести все попарные расстояния между двумя наборами векторов. Поэтому аргументы X и Y могут быть матрицами.

Если X  - набор векторов длины m (X.shape[0]==m), в Y - набор векторов длины n (Y.shape[0]==n), то на выходе получаем матрицу расстояний размерности (m x n).

#### Jaccard

Меру Жаккара можно посчитать функцией [sklearn.metrics.jaccard_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html).

Эта метрика не из модуля sklearn.metrics.pairwise, поэтому ее можно посчитать только для пары векторов.


## Поиск ближайших соседей

Попробуем вывести все попарные расстояния для первых 100 документов нашей выборки.

In [6]:
# Переводим тексты в вектороное представление
vect = TfidfVectorizer()
vectorized_text = vect.fit_transform(text_train['data'])

# Вычисляем матрицу расстояний
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(vectorized_text[0:100], vectorized_text[0:100])
sim_matrix.shape

(100, 100)

Проверим рассчитанные значения косинусного расстояния. 

In [7]:
sim_matrix[0,:]

array([1.        , 0.03673497, 0.05673791, 0.0174853 , 0.0429553 ,
       0.04380125, 0.028339  , 0.03478446, 0.02329201, 0.0265978 ,
       0.06227516, 0.05040917, 0.05015605, 0.02212076, 0.02852566,
       0.01073885, 0.02376148, 0.01664064, 0.02358362, 0.02110355,
       0.01595373, 0.03403637, 0.02214675, 0.0152878 , 0.07751104,
       0.0116598 , 0.01095673, 0.020069  , 0.01312511, 0.03081208,
       0.05077404, 0.04798343, 0.02879682, 0.03533289, 0.03344134,
       0.01479038, 0.026262  , 0.03299799, 0.04018106, 0.0253074 ,
       0.01686755, 0.0305604 , 0.02093238, 0.02697748, 0.03021066,
       0.04073356, 0.03039813, 0.05718717, 0.01759402, 0.02020483,
       0.09577765, 0.0203808 , 0.02011274, 0.03864396, 0.02860147,
       0.05271046, 0.03936306, 0.06122792, 0.02305622, 0.03975851,
       0.0463685 , 0.02608623, 0.00614942, 0.03228326, 0.01577141,
       0.01938515, 0.03979195, 0.01991409, 0.0398553 , 0.03307226,
       0.0398576 , 0.01882212, 0.01662219, 0.03276784, 0.05187

Видим, что на диагонали всегда стоит 1.0 - совпадение документа с самим собой всегда максимально. Какие-то пары документов больше похожи друг на друга, какие-то меньше. 

Имея матрицу расстояний, можем для любого текста найти наиболее похожие на него тексты.
Такое действие может лежать в основе например:
- системы поиска дубликатов на сайте объявлений
- системы определения плагиата в научных работах
- системы рекомендаций статей в агрегаторе изданий
- системы рекмендаций фильмов, содержание которых описываются некоторым текстом

Выведем номера первых пяти документов, наиболее сходных с запрашиваемым:

In [5]:
# Для какого текста ищем соседей
query_document_num = 10

# Расстояния до остальных текстов
query_neighbors = sim_matrix[query_document_num,:]
query_neighbors

# Выводим индексы документов с 5 наименьшими расстояниями
sorted(range(len(query_neighbors)), key=lambda x: query_neighbors[x], reverse=True)[0:5]

NameError: name 'sim_matrix' is not defined

# Кластеризация текстов



Тексты можно использовать в качестве признаков и в __unsupervised__ задачах, например, в задаче кластеризации.

Типичные кейсы:
- клиенты пишут заявки в службу техподдержки; в рамках анализа статистики запросов нужно определить основные темы обращений 
- покупатели оставляют отзывы на товар; основные темы отзывов
- новостной агрегатор; необходимо сгруппировать статьи в тематические блоки и отфильтровать дубликаты

Вообще, векторное представление характерно не только для текстов, так могут представляться любые наборы категорийных атрибутов. 

Примеры задач:
- анализируется аудитория посетителей веб-сайта, и каждый посетитель описывается с помощью своего поведенческого профиля (например, интерес "спорт":1, интерес "путешествия":10, интерес "машины" :6); в рамках задачи персонализации стратегии коммуникации с клиентами необходимо разбить клиентскую базу на 5 однородных кластеров
- сгруппровать пользователей по списку скачанных приложений в несколько кластеров, описать каждый и по каждому рекомендоватьсвой тарфиный план

### Предобработка

Для кластеризации загрузим выборку из датасетов Sklearn (тексты из тематических групп)

In [ ]:
categories_to_load = ['sci.space','talk.politics.mideast','rec.autos']
from sklearn.datasets import fetch_20newsgroups
text_dataset = fetch_20newsgroups(subset='train', categories=categories_to_load, shuffle=True)

Сообщения там подобного вида

In [ ]:
print(text_dataset['data'][1])

Помним, что для работы с текстами, их сначала нужно представить в виде векторов. Для этого используем уже знакомый TfidfVectorizer.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
vectorized_text = vect.fit_transform(text_dataset['data'])

In [ ]:
len(vect.vocabulary_)

Давайте сделаем обратное преобразование: посмотрим, какие слова закодированы в первом предложении. Для этого выделим ненулевые каунты из получившейся document-term матрицы.

In [ ]:
nonzero_dimensons = numpy.nonzero(vectorized_text[0])
print(nonzero_dimensons)

Координаты ненулевых элементов возвращаются двумя списками - сначала координаты строк, затем координаты стоблцов. Мы взяли только первую строку Document-Term матрицы, поэтому в первом списке везде 0. Теперь хотим координаты первести в соотвестующие им слова.

Словарь, который мы заполнили, вызывая метод fit(), имеет формат {term => dimension}. Но нам наоборот нужно по номеру измерения достать кодируемое слово {dimension=>term}. Воспользуемся одним питоновским финтом, чтобы инвертировать словарь.

In [ ]:
import numpy
inverse_vocabulary = {v:k for k,v in vect.vocabulary_.items()}

Теперь словарь выглядит так (возьмем 10 записей из середины)

In [ ]:
print(list(inverse_vocabulary.items())[10000:10010])

и можно применить функцию поиска в словаре по ключу (метод get) к нашему списку непустых элементов. Список слов выглядит так (первые 20 штук):

In [ ]:
list(map(inverse_vocabulary.get, list(nonzero_dimensons[1])))[0:20]

Или удобно использовать уже инвертированный словарь, который разработчики добавили в класс TfidfVectorizer. Результат тот же.

In [ ]:
# [vect.get_feature_names()[x] for x in nonzero_dimensons[1]][0:20]
# vect.vocabulary_
vect.get_feature_names()http://localhost:8888/notebooks/git/lesson2/NLP_lesson2.ipynb#
# nonzero_dimensons[1]

Всегда рекомендуется устанавливать параметры min_df и max_df. Это сильно сократит размерность, а также избавит от неифномартивных термов.

Параметры можно ставить как доли от общего числа документов в корпусе, либо как абсолютные значения.

Оптимальные пороги определяются методом проб - либо с помощью GridSearchCV, либо визуально при исследовании результатов.

In [ ]:
vect = TfidfVectorizer(min_df=0.05, max_df=0.5)
vectorized_text = vect.fit_transform(text_dataset['data'])
len(vect.vocabulary_)

In [ ]:
vect.vocabulary_

## Кластеризация алгоритмом K-means

Попробуем применить наиболее популярный алгоритм кластеризации K-means.

K-means - итеративный алгоритм. Сходимость алгоритма проиллюстрирована на картинке ниже:
<img src="img/kmeans.png" width=700>

Алгоритм следующий:
1. Случайным образом выделяем центры наших 2 кластеров (b)
2. Относим каждую точку в тот кластер, к центру которого она ближе (c)
3. Пересчитываем центры внось полученных кластеров (d) и повторяем с пункта 1

Через некоторе кол-во шагов алгоритм сойдется к решению.

NOTE: как это очень часто бывает в машинном обучении, из-за случайного первого приближения, результаты могут различаться от запуска к запуску. Нужно всегда иметь это в виду.

Обратите внимание, что в k-means, как и в большинстве алгоритмов кластерзиации, число кластеров задается вручную до запуска алгоритма:
- n_clusters

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(vectorized_text)

Выведем номер кластера, в который относим точку:

In [ ]:
kmeans.labels_

Вывести центры кластеров

In [ ]:
kmeans.cluster_centers_

### Описание кластеров

Отдельная задача, выполняемая в рамках кластерзиации - **описание полученных кластеров**.

Один из способов - посчитать метрику **uplift**, то есть насколько чаще слово встречается в рамках данного кластера, чем в остальных кластерах. И затем по каждому кластеру вывести набор слов с наибольшим аплифтом. Эти слова и будут лучше всего выделять данную группу текстов.

$$ \text{Uplift(w,}C_{k}\text{)} = \frac{|w_{k}|}{|C_{k}|} / \frac{|w_{other}|}{|C_{other}|}$$

где:
- $|w_{k}|$ - число документов в кластере $C_k$, в которых встречается слово $w$
- $|w_{other}|$ - число документов в остальных кластерах, в которых встречается слово $w$
- $|C_{k}|$ - число документов в кластере $C_k$
- $|C_{other}|$ - число документов в остальных кластерах

## Topic Modeling

Моделирование тематики текста - другой пример обучения без учителя. Основное отличие от кластерзиации - текст не обязательно относится только к одному из кластеров, он может содержать сразу нескольких тематик. 

На практике бывают ситуации, когда кластеризация дает слишком глубое решение, особенно если нет одной ярко выраженной темы, но есть комбинация нескольких тематик. В этих случаях прибегают к тематическому моделированию.

[Latent Semantic Analysis ](https://en.wikipedia.org/wiki/Latent_semantic_analysis) (LSA) - общее название для методов выделения тематик в тексте.

#### Почнму семантический?
Потому что цель анализа - выделение смысла (семантики) в тексте

#### Почему латентный?
В машинном обучении так называют ситуации, когда при моделировании какого-то процесса присутствует некая ненаблюдаемая (латентная) переменная, которая объясняет суть процесса, но о ее конкретных значениях мы можем только делать предположения. В случае с текстами такой латентной переменной обычно называют как раз тематику текста - она влияет на то, какие слова используются в тексте, но мы её не знаем. Задача LSA как раз сделать предположение относительно тематики.

#### Теоретическое введение
Любой сложный сигнал (в том числе временные ряды, картинки, текст) можно представить, как комбинацию нескольких крупных сигналов и большое кол-во более мелких. Здесь работает такая аналогия - когда художник рисует каритину, он сначала обозначает горизонт, крупными мазками закрашивает небо, землю, деревья, затем добавляет объекты среднего размера, и уже затем прорабатывает все мелкие детали. Если мы научимся декомпозировать картину на составляющие ее сигналы, то сможем на нее смотреть с любым уровнем детализации.

По такому принципу работает поиск похожих картинок в Google - алгоритм не сравнивает картинки попиксельно, он сначала определяет семантику, примерное содержание изображенного на картинке, а уже потом ищет в своей базе картинки с похожим смысловым наполнением.

По этому же принципу работает, например, Google переводчик. Он не лезет в словарь за переводом каждого слова, он сначала определяет смысл (семантику) сказанного, а уже потом преобразует этот смысл в текст на другом языке.

То есть это все методы, заточенные на определение семантики (смыслового содержание) сигнала, а не его описания.

Технически почти все эти алгоритмы реализованы так - они ищут некое сокращенное описание сигнала, которое сохраняет максимум информации о нем. Это сокращенное (латентное) описание и есть семантика сигнала.

#### Применительно к текстам

Нам хочется, чтобы текст представлялся не как набор слов, а как комбинация небольшого числа осмысленных характеристик. Например, "текст об искусстве; речь от первого лица; есть ссылки на факты". 

Простые модели (SVD, pLSA, LDA) позволяют выделять тематики, составляющие текст. Более сложные модели, основанные на нейронных сетях, опредляют не только тематику, но могут смотреть глубже: например, ведется ли речь от первого лица, сколько в повествовании главных геороев и прочее. На этом занятии ограничимся рассмотрением тематических моделей.

В математике есть понятие "спектральное разложение матриц". Суть в том, что любую матрицу можно выразить как произведение других матриц, причем точность приближения можно выбирать любую, просто варьируя размерность соствляющих матриц.

В нашем случае матрица - это Document-Term матрица, описывающая наши тексты. Если мы ее разложим в произведение более простых матриц, получим скоращенное описание текстов. Такое разложение выделяет только значимый сигнал, отсеивая малозначимый шум. По этой причине процесс описанный в этом разделе, ещё иногда называют низкочастотной фильтрацией. 

К сожалению мы не знаем, как именно интерпретируются элементы сокращенного описания, но частично можем ответить на этот вопрос, выведя топ слов, обладающих наибольшим весом в описании каждого элемента (тематики).

Таким образом, на выходе имеем:
- описание найденных в корпусе тематик через топ слова
- представлем все документы в виде векторов размерности сильно меньшей оригинальной => это позволяет эффективно использовать эти данные при дальнейших манипуляциях с текстами: решения задач классификации, кластеризации, поиска дубликатов.

Посмотрим, какие есть методы разложения

### Singular-Value Decomposition

Взятое из математики матричное сингулярное разложение (SVD) очень популярно в машинном обучении. Оно раскладывает произвольную прямоугольную матрицу в произведение 3 матриц. 

<img src="img/svd.png" width=500>

Причем это разложение можно делать приближенным, просто уменьшая ранг **r** - вторую размерность матриц U и V. 
- Если оставляем r=n, то имеем дело с обычным равенством
- Если ставим r меньше, равенство заменяется на приближенное равенство
- Чем меньше ставим ранг, тем грубее получается приближение, но зато тем проще можем описывать наши тексты

SVD - удобный интсрумент получения сокращенного латентного описания текстов

Зачем вообще это нужно:
1. получаем отдельное представление для "строк" и "столбцов" матрицы A
    
        применительно к нашей задаче строки - тексты, столбцы - слова => получаем сокращенное описание текстов и сокращенное описание слов


2. можем выбрать любую размерность k получаемого представления "строк" и "столцов", обычно k << n

        чем меньше k, тем "грубее" описание каждого текста, но тем более оно сжатое =>
            - имея компактные описания мы можем более эффективно работать с данными
            - тем меньше в описаниях случайного шума, а шум в задачах ML - это плохо
    
3. перемножая матрицы обратно, можем приблизить исходную матрицу с любой наперед выбранной точностью

        и тем самым можем, например, оценить, насколько слово соотвествует данному документу, несмотря на то, что в орининальном тексте этого слова нету

### TruncatedSVD
В библиотеке sklearn за это отвечает класс TruncatedSVD. Здесь преобразование называется "Truncated", потому что его смысл в получении описаний меньшего размера (r < n).

In [ ]:
from sklearn.decomposition import TruncatedSVD

Основной параметр: 
- n_components -  ранг разложения

In [ ]:
svd = TruncatedSVD(n_components=3)

Разложение выполняется методом fit_transform()

На выходе получаем уменьшенное описание документа в виде неинтерпретируемого вектора фиксированной размерности

In [ ]:
reduced_vectorized_text = svd.fit_transform(vectorized_text)
reduced_vectorized_text

In [ ]:
reduced_vectorized_text.shape

Мы устанавливали размерность равную 10, видим, что матрица трансформировалась в матрицу размерности (1741 x 10)

In [ ]:
categories_to_load = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
text_dataset = fetch_20newsgroups(subset='train', categories = categories_to_load, shuffle=True)

vect = TfidfVectorizer(min_df=0.05, max_df=0.5)
vectorized_text = vect.fit_transform(text_dataset['data'])

### Probabilistic LSA (pLSA)
Модификацией описанного выше подхода LSA является "Probabilistic LSA" (pLSA). Здесь пытаемся описать с помощью вероятностной модели.

В рамках этой модели каждое слово генерируется с некоторой веротяностью. 
1. Вероятностное распределение слова зависит от выбранной для этого слова тематики 
2. Тематика слова зависит от того, что это за документ (в каждом документе свое распределение тематик)

### Latent Dirichlet Allocation (LDA)
В своей базовой постановке метод pLSA почти не используется. Наиболее часто встречающаяся реализация pLSA - это метод Latent Dirichlet Allocation (LDA). Называется так потому, что в качестве распределения тематики в документе P(c|d) и слова в тематике P(w|c) используются [распределения Дирихле](https://en.wikipedia.org/wiki/Dirichlet_distribution).

### Реализация в Sklearn

В Sklearn для этого есть класс LatentDirichletAllocation.

In [25]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=4,
                                learning_method='online',
                                learning_offset=50.)

Мы явно указываем, что в тексте ожидаем 4 темы (n_components=4). В результате хотим увидеть список тем, каждую как взвешенную сумму термов.

Обучение происходит методом fit()

In [ ]:
lda.fit(vectorized_text)

Атрибут components_ возвращает веса кадого терма для каждого топика:

In [ ]:
lda.components_

Чтобы как-то описать выделенные тематики, выведем по несколько слов, наиболее ассоциированных с каждой из них:

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
print_top_words(lda, vect.get_feature_names(), 10)

В описаниях какие-то тематики угадываются легко, какие-то не очень. 

Обратите внимание, что тематики могут выводиться в произвольном порядке между разными запусками модели, поэтому не стоит их называть по номеру.

In [ ]:
categories_to_load = ['rec.autos','rec.motorcycles']
text_dataset = fetch_20newsgroups(subset='train', categories = categories_to_load)

vect = TfidfVectorizer(min_df=0.05, max_df=0.25, ngram_range=(1,1), stop_words='english', use_idf=False)
vectorized_text = vect.fit_transform(text_dataset['data'])

lda = LatentDirichletAllocation(n_components=5,
                                learning_method='online',
                                learning_offset=50.)

lda.fit(vectorized_text)

print_top_words(lda, vect.get_feature_names(), 10)

# Математика методов Topic Modeling  (для интересующихся)

Рассмотрим систему из нескольких случайных величин, какие-то из которых могут зависеть друг от друга.

Вероятностная графическая модель - способ визуализации подобных зависимостей в виде графа.

Например, здесь представлены 5 случайных величин, но каждая зависит от другой. Оценка (Grade) зависит от сложности экзамена (Difficulty) и успеваемости студента (Intelligence). Средний бал (SAT) зависит от успеваемости в течение года. А наличие рекомендательного пиьсма (Letter) зависит от оценок за экзамен.

<img src="img/probabilistic.jpeg" width=350>

## Plate Notation

Plate Notation - удобный способ схематичного изображения графических веротяностных моделей, которые содержат большое число повторяющихся элементов.

Цель: удобство восприятия модели

Как читается Plate Notation?
- Стрелка - наличие зависимости (переменная в которую идет стрелка зависит от первой переменной)
- Закарашенные круги - наблюдаемые переменные
- Пустые - латентные переменные
- Прямоугольник с числом N внизу - означает "повторяется N раз"

Проиллюстрируем Plate Notation на примере тематической модели pLSA

<img src="img/plate0.png" width=300>



У нас есть M документов (внешняя плашка) и в каждом таком документе N слов (внутренняя). 

С каждым словом связана латентная переменная c, обозначающая тематику. Веротяность тематики зависит от документа: P(c|d), так как каждый документ это какой-то свой микс из нескольких тематик.

А распределение конкретного слова w зависит от выбранной тематики P(w|c).

**Задача pLSA:** найти конкретные распределения P(c|d) и P(w|c).

У нас есть корпус текстов, поэтому подбираем наиболее правдоподобную модель (распределения) мы методом максимального правдоподобия.

-----

## Dirichlet Distribution

Распределение Дирихле  - вероятностное распределение k-элементного вектора $x=(x_1,x_2 \dots x_k)$ при условии $\sum{x_i}=1$.

В математике множество векторов ($x_1 ... x_k$), где $\sum{x_i}=1$ называют <a href="https://ru.wikipedia.org/wiki/%D0%A1%D0%B8%D0%BC%D0%BF%D0%BB%D0%B5%D0%BA%D1%81">симплексом</a>, он же (k-1) -мерный треугольник. 

То есть распределение Дирихле - это просто распределение вероятности на неком гипертреугольнике. На картинке ниже пример для k=3:
<img src="img/dirichlet.png" width=350>

Плотность распределния Дирихле выражается через произведение координат $x_i$, нормированное Бета-функцией: $$f \left(x_1,\ldots, x_{K}; \alpha_1,\ldots, \alpha_K \right) = \frac{1}{\mathrm{B}(\boldsymbol\alpha)} \prod_{i=1}^K x_i^{\alpha_i - 1}$$

### Параметры распределения

У распределения Дирихле есть параметры $\hat{\alpha} = (\alpha_1, \alpha_2, \dots \alpha_k)$, задающие конкретный вид распределения: $\alpha_i > 0$.

Посмотрим, как меняется распределение при разных параметрах. Для наглядности возьмем k=3.

|$\bar{\alpha}$|распределение|
|---|---|
|(1,1,1) | равномерное распределение |
|(0.2,0.2,0.2) | больше вероятности у краев (когда часть $x_i$ зануляется)|
|(7,7,7) | больше распределения у центра |
|(5,5,2)| сдвигается к одной из вершин |

<img src="img/dir1.png" width=300>

Давайте посэмплируем точки из трех распределений Дирихле:
- (0.1,0.1,0.1)
- (1,1,1)
- (4,4,4)

Получится примерно следующее:

<img src = "img/dir2.png" siwth=250>

То есть видим, что распределение Дирихле позволяет довольно гибко моделировать различные смеси тематик / смеси слов, поэтому и стали использовать в методе.

-----


В рамках LDA используется байесовский способ подбора параметров модели. Это отдельная сложная тема. Суть: сначала мы выбраем некое априорное распределение для параметров (простое), а затем итеративно корректируем его с учетом наблюдаемых данных выборки.

В качестве априорных распределений для $\alpha$ и $\beta$ используются также распределения Дирихле. Причем используется симметричное распределение 
- $\alpha = \alpha_1 = \alpha_2 = \dots = \alpha_k$ 
- $\beta = \beta_1 = \beta_2 = \dots = \beta_k$

LDA в plate нотации выглядит так: 
<img src="img/lda.png">

Как с точки зрения модели пошагово выглядит процесс, генерирующий корпус текстов?
1. Для каждого документа генерируем своё распределение тематик  $\theta_i \sim Dir(\alpha)$
2. Для каждой тематики генерирем распределение слов $\phi_j \sim Dir(\beta)$
3. Для каждой позиции в документе выбираем тематику $z_{ij} \sim Multinomial(\theta_i)$
4. Из этой тематики выбираем конкретное слово, которое ставится в эту позицию $w_{ij} \sim Multinomial(\phi_{z_{ij}})$

## Меры качества в Topic Modeling

### Perplexity 

Perplexity - среднее правдоподобие модели:

$$P=\exp\bigg[ -\frac{\sum{\log{P(w|\alpha,\beta)}}}{\sum{N_d}}\bigg] = \exp^{-E \big[ \log P(w|\alpha, \beta) \big]}$$

где $w$ - слово, $\alpha, \beta$ - наденные параметры модели, $N_d$ - кол-во слов в документе d, всего M документов

Arxiv: <a href="http://dirichlet.net/pdf/wallach09evaluation.pdf">Wallach09</a>

$P \in [0, +\inf)$

- Если имеем полную разделимость по тематикам и средняя $P(w|\alpha,\beta) = 1$, то $P = 0$
- Если всё перемешано и средняя $P(w|\alpha,\beta) = 0$, то $P = 1$

Разумеется, perplexity уменьшается с ростом числа топиков, но можно использовать для подбора других параметров:
<img src="img/perplexity1.png" width = 500>

Здесь например приведена сетка из 2 параметров модели LDA: num_topics и learning_decay

### Coherence

Когерентность тематик - группа метрик, считающих "целостность" топика по словам, его сотавляющим.

Первые метрики были просто расстоянием до плохих topic-word распределений (junk topics). К плохим тематикам можно отнести  те, в которых:
- все слова равноверотяны
- распределение слов в тематике совпадает с распределением во всем корпусе текстов
- тематика распределена равноверотяно по всем документам

Метрики качества топиков можно разделить на
- Extrinsic - которые требуют внешнего датасета для расчета
- Intrinsic - не требуют

Наиболее распространены две метрики: 
- UMass (intrinsic)
- UCI (extrinsic)

Обе сначала считаются для каждой пары слов (pairwise), составляющих тематику, а затем суммируются:

$$Coherence = \sum_{i<j}{score(w_i,w_j)}$$

Arxiv: <a href="http://dirichlet.net/pdf/mimno11optimizing.pdf">Wallach09</a>

UCI - это Pointwise Mutual Information для пары слов w_i и w_j

$$UCI = \log{\frac{P(w_i,w_j)}{P(w_i) \cdot P(w_j)}}$$

UMass

$$UMass = \log{\frac{P(w_i,w_j)+1}{P(w_j)}}$$

В Gensim для расчета когерентности топика используют более продвинутый подход, основанный на алгоритме описанном в <a href="http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf">WSDM_Topic_Evaluation</a>

<img src="">

## References
[Topic Modeling Review](https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05) - good article on medium

[LDAviz](https://github.com/TomasKeller/python-LDAviz) - nice python library for topic modeling with good visualization capabilities, almost no tuning requrired

[Machinelearningmastery](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) - simple example of text generation using neural networks and very good blog on ML
